In [1]:
# importing tools 
import s3fs
import xarray as xr
import numpy as np 
import matplotlib.pyplot as plt 
import math
#for removing warnings
import warnings

In [2]:
# open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path = 's3://velocity-u.zarr/'
remote_files = fsg.glob(data_path)

store = s3fs.S3Map(root=data_path, s3=fsg, check=False)
du = xr.open_zarr(store=store, consolidated=True)
du = du.rename({'depthu' : 'depth'})
du 

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, depth: 75, time_counter: 73, x: 1580, y: 1801)
Coordinates:
  * depth                 (depth) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    nav_lat               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    time_centered         (time_counter) object dask.array<chunksize=(73,), meta=np.ndarray>
  * time_counter          (time_counter) object 2000-01-03 12:00:00 ... 2000-...
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    depthu_bounds         (time_counter, depth, axis_nbounds) float32 dask.array<chunksize=(1, 75, 2), meta=np.ndarray>
    sozotaux              (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) object dask.array<chunksize=(73, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) object dask.array<chunksize=(73, 2), meta=np.ndarray>
    utau_atmoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    utau_iceoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    uwspd10               (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    vozocrtx              (time_counter, depth, y, x) float32 dask.array<chunksize=(1, 75, 1801, 1580), meta=np.ndarray>
Attributes:
    CASE:              REF08
    CONFIG:            CREG12.L75
    Conventions:       CF-1.5
    description:       ocean U grid variables
    name:              /scratch/cnt0028/lpo7420/talandic/CREG12.L75-REF08-XIO...
    output_frequency:  5d
    production:        An IPSL model
    start_date:        19790101
    timeStamp:         2020-May-14 08:39:48 CEST
    title:             ocean U grid variables

In [3]:
# open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path = 's3://velocity-v.zarr/'
remote_files = fsg.glob(data_path)

store = s3fs.S3Map(root=data_path, s3=fsg, check=False)
dv = xr.open_zarr(store=store, consolidated=True)
dv = dv.rename({'depthv' : 'depth'})
dv

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, depth: 75, time_counter: 73, x: 1580, y: 1801)
Coordinates:
  * depth                 (depth) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    nav_lat               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    time_centered         (time_counter) object dask.array<chunksize=(73,), meta=np.ndarray>
  * time_counter          (time_counter) object 2000-01-03 12:00:00 ... 2000-...
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    depthv_bounds         (time_counter, depth, axis_nbounds) float32 dask.array<chunksize=(1, 75, 2), meta=np.ndarray>
    sometauy              (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) object dask.array<chunksize=(73, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) object dask.array<chunksize=(73, 2), meta=np.ndarray>
    vomecrty              (time_counter, depth, y, x) float32 dask.array<chunksize=(1, 75, 1801, 1580), meta=np.ndarray>
    vtau_atmoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    vtau_iceoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    vwspd10               (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
Attributes:
    CASE:              REF08
    CONFIG:            CREG12.L75
    Conventions:       CF-1.5
    description:       ocean V grid variables
    name:              /scratch/cnt0028/lpo7420/talandic/CREG12.L75-REF08-XIO...
    output_frequency:  5d
    production:        An IPSL model
    start_date:        19790101
    timeStamp:         2020-May-14 08:39:49 CEST
    title:             ocean V grid variables

In [4]:
def open_s3fs(path):
    # open the data with s3fs from the databucket 
    fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
    data_path = f's3://data/{path}/*'
    remote_files = fsg.glob(data_path)
    fileset = [fsg.open(file) for file in remote_files]
    #open the dataset 
    dset = xr.open_mfdataset(fileset, combine='by_coords',compat='override')
    # rename variabel
    dset = dset.rename({'__xarray_dataarray_variable__' : 'vel'})
    return dset

In [5]:
ds = open_s3fs('PC-data')

In [6]:
u = du.vozocrtx.isel(y=slice(900,1800), x=slice(200,1400))
v = dv.vomecrty.isel(y=slice(900,1800), x=slice(200,1400))
mbathy = ds.mbathy.isel(y=slice(900,1800), x=slice(200,1400))

# Ser på U data alene

Da har jeg 73 variabler. 
Om jeg er på U og V har jeg 2*73=146 variabler

### Loopet over hele området 

Dette skal per def funke, men stort dataset og lite minne er det ikke sikkert det går.. 

### Note til meg selv: 
Hva ønsker jeg å finne?
- PC1 i verticalen for alle punkter i griddsystemet? Eller flere PCX også?
- Bare eigenvalues siden vi ikke trenger retningen? 


In [19]:
'List over the fist prinsipal components for all depths '
PC11 = []
PC12 = []
dybde = []
x_l = []
y_l = []

'does this make the loop go faster?'
lenx = np.arange(0,1200)
leny = np.arange(0,900)

'limit for how short the depth matrix can be'
limit = np.zeros((73,2))

PC11 = np.zeros((1200,900))
PC12 = np.zeros((1200,900))
x_l = np.zeros((1200,900))
y_l = np.zeros((1200,900))
dybde = np.zeros((1200,900))

'Looping over the area, x,y '
for x in lenx:
    for y in leny:
        d = mbathy.isel(x=x,y=y).values # henter nivået for bunn i punktet
        
        if d<= 2:
            PC11[x,y] = np.nan
            PC12[x,y] = np.nan
            x_l[x,y] = np.nan
            y_l[x,y] = np.nan
            dybde[x,y] = np.nan
        else:
            'Making a U-matrix with all 73 timestep as variables and all depths as samples'
            dyp = u.isel(x=x, y=y, depth = slice(0,d-1,10)) # -1 pga d=0 og vi vil en opp der vi har bunnhastigheter
            
            if dyp.shape < limit.shape:
                PC11[x,y] = np.nan
                PC12[x,y] = np.nan
                x_l[x,y] = np.nan
                y_l[x,y] = np.nan
                dybde[x,y] = np.nan
            else: 
                cov_d = np.cov(dyp.T, bias=True) # uten .T ble den en 73x73 matrise, vil ha depthxdepth
                values, vectors = np.linalg.eig(cov_d)
                order = values.argsort()[::-1]
                values, vectors = values[order], vectors[:, order]

            
            '''
            PC11.append(vectors[0,0]*np.sqrt(values[0]))
            PC12.append(vectors[1,0]*np.sqrt(values[0]))
            dybde.append(d-1)
            x_l.append(x)
            y_l.append(y)
            '''
            
            PC11[x,y] = vectors[0,0]*np.sqrt(values[0])
            PC12[x,y] = vectors[1,0]*np.sqrt(values[0])
            x_l[x,y] = x
            y_l[x,y] = y
            dybde[x,y] = d-1
            
        print('y=', y, ' ferdig')
    print('x=', x, ' ferdig')


y= 0  ferdig
y= 1  ferdig
y= 2  ferdig
y= 3  ferdig
y= 4  ferdig
y= 5  ferdig
y= 6  ferdig
y= 7  ferdig
y= 8  ferdig
y= 9  ferdig
y= 10  ferdig
y= 11  ferdig
y= 12  ferdig
y= 13  ferdig
y= 14  ferdig
y= 15  ferdig
y= 16  ferdig
y= 17  ferdig
y= 18  ferdig
y= 19  ferdig
y= 20  ferdig
y= 21  ferdig
y= 22  ferdig
y= 23  ferdig
y= 24  ferdig
y= 25  ferdig
y= 26  ferdig
y= 27  ferdig
y= 28  ferdig
y= 29  ferdig
y= 30  ferdig
y= 31  ferdig
y= 32  ferdig
y= 33  ferdig
y= 34  ferdig
y= 35  ferdig
y= 36  ferdig
y= 37  ferdig
y= 38  ferdig
y= 39  ferdig
y= 40  ferdig
y= 41  ferdig
y= 42  ferdig
y= 43  ferdig
y= 44  ferdig
y= 45  ferdig
y= 46  ferdig
y= 47  ferdig
y= 48  ferdig
y= 49  ferdig
y= 50  ferdig
y= 51  ferdig
y= 52  ferdig
y= 53  ferdig
y= 54  ferdig
y= 55  ferdig
y= 56  ferdig
y= 57  ferdig
y= 58  ferdig
y= 59  ferdig
y= 60  ferdig
y= 61  ferdig
y= 62  ferdig
y= 63  ferdig
y= 64  ferdig
y= 65  ferdig
y= 66  ferdig
y= 67  ferdig
y= 68  ferdig
y= 69  ferdig
y= 70  ferdig
y= 71  ferdig
y=

KeyboardInterrupt: 

# Tester for et punkt først: 

Trenger å finne ut hvor langt ned i dypet punktet har verdier

In [7]:
x_p = 500
y_p = 500 

'Finding the level the given location dosent have values for anymore'

mbathy = ds.mbathy.isel(y=slice(900,1800), x=slice(200,1400))
d = mbathy.isel(x=x_p, y=y_p).values

dyp = u.isel(x=x_p, y=y_p, depth = slice(0,d-1)) # -1 pga d=0 og vi vil en opp der vi har bunnhastigheter

cov_p = np.cov(dyp.T, bias=True) # uten .T ble den en 73x73 matrise 
values, vectors = np.linalg.eig(cov_p)
order = values.argsort()[::-1]
values, vectors = values[order], vectors[:, order]

values[0]    

0.000772891711862852

In [10]:
u

<xarray.DataArray 'vozocrtx' (time_counter: 73, depth: 75, y: 900, x: 1200)>
dask.array<getitem, shape=(73, 75, 900, 1200), dtype=float32, chunksize=(1, 75, 900, 1200), chunktype=numpy.ndarray>
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    nav_lat        (y, x) float32 dask.array<chunksize=(900, 1200), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(900, 1200), meta=np.ndarray>
    time_centered  (time_counter) object dask.array<chunksize=(73,), meta=np.ndarray>
  * time_counter   (time_counter) object 2000-01-03 12:00:00 ... 2000-12-29 1...
Dimensions without coordinates: y, x
Attributes:
    cell_methods:        time: mean (interval: 360 s)
    interval_operation:  360 s
    interval_write:      5 d
    long_name:           sea_water_x_velocity
    online_operation:    average
    standard_name:       sea_water_x_velocity
    units:               m/s

In [55]:
len(u.y)

900

In [9]:
x_p = 500
y_p = 500
mbathy = ds.mbathy.isel(y=slice(900,1800), x=slice(200,1400))
mbathy

<xarray.DataArray 'mbathy' (y: 900, x: 1200)>
dask.array<getitem, shape=(900, 1200), dtype=int16, chunksize=(900, 1200), chunktype=numpy.ndarray>
Coordinates:
    time_centered  object ...
    time_counter   object ...
    nav_lat        (y, x) float32 dask.array<chunksize=(900, 1200), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(900, 1200), meta=np.ndarray>
Dimensions without coordinates: y, x

In [45]:
d = mbathy.isel(x=500, y=500).values
print(d)
d[d==-1] = 0
#print(d)


61
61


In [39]:
dyp = u.isel(x=x_p, y=y_p, depth = slice(0,d))

In [42]:
dyp

<xarray.DataArray 'vozocrtx' (time_counter: 73, depth: 61)>
dask.array<getitem, shape=(73, 61), dtype=float32, chunksize=(1, 61), chunktype=numpy.ndarray>
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 2.956e+03 3.139e+03
    nav_lat        float32 dask.array<chunksize=(), meta=np.ndarray>
    nav_lon        float32 dask.array<chunksize=(), meta=np.ndarray>
    time_centered  (time_counter) object dask.array<chunksize=(73,), meta=np.ndarray>
  * time_counter   (time_counter) object 2000-01-03 12:00:00 ... 2000-12-29 1...
Attributes:
    cell_methods:        time: mean (interval: 360 s)
    interval_operation:  360 s
    interval_write:      5 d
    long_name:           sea_water_x_velocity
    online_operation:    average
    standard_name:       sea_water_x_velocity
    units:               m/s

In [54]:
len(dyp.depth)

61

In [47]:
cov_p = np.cov(dyp, bias=True)

In [49]:
cov_p.shape

(73, 73)

In [50]:
values, vectors = np.linalg.eig(cov_p)
order = values.argsort()[::-1]
values, vectors = values[order], vectors[:, order]

print(values[0])

(0.003562873494877785+0j)


In [51]:
print(values[1])

(0.0008212884899316445+0j)


In [52]:
len(values)

73

In [56]:
i_list = np.arange(0,len(values),20)
j_list = np.arange(0,len(values),20)

In [57]:
i_list

array([ 0, 20, 40, 60])

In [60]:
for i in range(len(i_list)):
    print(i) 

0
1
2
3


In [34]:
# Getting the bottom velocity 
d = ds.mbathy[900:1800,200:1400].values-1
d[d==-1] = 0

d = xr.DataArray(d, dims=['y','x'])


NameError: name 'uvar' is not defined

In [12]:
dyp.shape

(73, 1)

In [17]:
limit = np.zeros((73,2))
limit.shape

(73, 2)